In [1]:
%%writefile strands_claude_streaming.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@tool
def get_time():
    """ Get current time """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id= "global.anthropic.claude-haiku-4-5-20251001-v1:0"

model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time."""
)

@app.entrypoint
async def strands_agent_bedrock_streaming(payload):
    """
    Invoke the agent with streaming capabilities
    This function demonstrates how to implement streaming responses
    with AgentCore Runtime using async generators
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    
    try:
        # Stream each chunk as it becomes available
        async for event in agent.stream_async(user_input):
            if "data" in event:
                yield event["data"]
            
    except Exception as e:
        # Handle errors gracefully in streaming context
        error_response = {"error": str(e), "type": "stream_error"}
        print(f"Streaming error: {error_response}")
        yield error_response

if __name__ == "__main__":
    app.run()

Overwriting strands_claude_streaming.py


In [2]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_streaming.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="pyproject.toml",
    region=region,
    agent_name="strands_claude_streaming"
)
response

Entrypoint parsed: file=/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/streaming-agent-response/strands_claude_streaming.py, bedrock_agentcore_name=strands_claude_streaming
Configuring BedrockAgentCore agent: strands_claude_streaming


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/streaming-agent-response/Dockerfile
Generated .dockerignore: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/streaming-agent-response/.dockerignore
Keeping 'strands_claude_streaming' as default agent
Bedrock AgentCore configured: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/streaming-agent-response/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/streaming-agent-response/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/streaming-agent-response/Dockerfile'), dockerignore_path=PosixPath('/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/streaming-agent-response/.dockerignore'), runtime='Docker', region='ap-southeast-2', account_id='381491838394', execution_role=None, ecr_repository=None, auto_create_ecr=True)

In [3]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_streaming' to account 381491838394 (ap-southeast-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_streaming
✅ ECR repository available: 381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-strands_claude_streaming
Getting or creating execution role for agent: strands_claude_streaming
Using AWS region: ap-southeast-2, account ID: 381491838394
Role name: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-2ce6205d24


✅ Reusing existing ECR repository: 381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-strands_claude_streaming


✅ Reusing existing execution role: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-2ce6205d24
✅ Execution role available: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-2ce6205d24
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_claude_streaming
Role name: AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-2ce6205d24
Reusing existing CodeBuild execution role: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-2ce6205d24
Using .dockerignore with 44 patterns
Uploaded source to S3: strands_claude_streaming/source.zip
Updated CodeBuild project: bedrock-agentcore-strands_claude_streaming-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 9.4s
🔄 DOWNLOAD_SOURCE started 

In [4]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_streaming


'READY'

In [5]:
invoke_response = agentcore_runtime.invoke({
    "prompt": 
    "what the weather is like?"
})
invoke_response

The

weather is

**

sunny**! It's

a nice

day out

there.

☀️

{}

Invoking AgentCore Runtime with boto3 for Streaming

In [15]:
import boto3
import json
from IPython.display import Markdown, display

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# For streaming responses, we need to handle the EventStream
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2+1"})
)

# Check if the response is streaming
if "text/event-stream" in boto3_response.get("contentType", ""):
    print("Processing streaming response with boto3:")
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                data = line[6:].replace('"', '')  # Remove "data: " prefix
                print(f"Received streaming chunk: {data}")
                content.append(data.replace('"', ''))
    
    # Display the complete streamed response
    full_response = " ".join(content)
    display(Markdown(full_response))
else:
    # Handle non-streaming response
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    
    if events:
        try:
            response_data = json.loads(events[0].decode("utf-8"))
            display(Markdown(response_data))
        except:
            print(f"Raw response: {events[0]}")

Processing streaming response with boto3:
Received streaming chunk: 2
Received streaming chunk:  + 1 = **3**


2  + 1 = **3**

In [16]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-strands_claude_streaming',
 'strands_claude_streaming-kgKwCS32Ir',
 'bedrock-agentcore-strands_claude_streaming')

In [17]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)